In [1]:
import gym
import time

In [2]:
env = gym.make('Pong-v4')

In [3]:
from stable_baselines3.common.callbacks import BaseCallback
import os

/home/alexandre/INSA/GM5/T2/PBIA/PBIA_Env/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self,check_freq,save_path, verbose=1):
        super(TrainAndLoggingCallback,self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path,exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path,'best_model_ppo_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [5]:
CHECKPOINT_DIR = './train/'

callback = TrainAndLoggingCallback(check_freq=50000, save_path = CHECKPOINT_DIR)

In [6]:
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack

In [7]:
LOG_DIR = './logs/'

In [8]:
# Permet d'extraire la recompense moyenne et la longueur moyenne d'un episode
env = Monitor(env,LOG_DIR)
#Nécessaire pour Stable Baselines
env = DummyVecEnv([lambda: env])
#Empile 4 vecteurs consécutifs
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
# Construction du modèle 
model = PPO('CnnPolicy',env,tensorboard_log=LOG_DIR,verbose=1)

model.learn(total_timesteps = 2500000, callback = callback)

Using cuda device
Wrapping the env in a VecTransposeImage.
Logging to ./logs/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.51e+03 |
|    ep_rew_mean     | -19      |
| time/              |          |
|    fps             | 117      |
|    iterations      | 1        |
|    time_elapsed    | 17       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.26e+03    |
|    ep_rew_mean          | -20.3       |
| time/                   |             |
|    fps                  | 52          |
|    iterations           | 2           |
|    time_elapsed         | 78          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007237888 |
|    clip_fraction        | 0.0307      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.